<a href="https://colab.research.google.com/github/mengcius/pytorch-learn/blob/master/15_%E8%BF%81%E7%A7%BB%E5%AD%A6%E4%B9%A0CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 15_迁移学习CIFAR10
ResNet18迁移学习cifar10

1、测试1 batchsz128，可能不适合cifar
0 loss: 1.0104621648788452
0 test acc: 0.7064

1 loss: 0.6776887774467468
1 test acc: 0.7688

2 loss: 0.8195689916610718
2 test acc: 0.7795

10 loss: 0.5256583094596863
10 test acc: 0.8258

22 loss: 0.24958983063697815
22 test acc: 0.8587

70 loss: 0.07972583919763565
70 test acc: 0.8629

99 loss: 0.09527455270290375
99 test acc: 0.8724

2、测试2 batchsz64，差不多一样

0 loss: 2.024899482727051
0 test acc: 0.7201

1 loss: 0.6208065748214722
1 test acc: 0.7446

2 loss: 1.001968264579773
2 test acc: 0.7767

10 loss: 3.493842601776123
10 test acc: 0.8219

19 loss: 0.3018224537372589
19 test acc: 0.8537

3、模拟验证集测试集 

epoch: 0 loss: 0.8336920738220215
epoch: 0 val acc: 0.7181

epoch: 1 loss: 1.0085989236831665
epoch: 1 val acc: 0.7728

epoch: 2 loss: 0.6742492318153381
epoch: 2 val acc: 0.7753

epoch: 3 loss: 0.5457809567451477
epoch: 3 val acc: 0.8057

epoch: 4 loss: 0.6048915982246399
epoch: 4 val acc: 0.8054

val best epoch: 3 val best acc: 0.8057

loaded from ckpt!

test acc: 0.8057

### 安装visdom

In [7]:
# !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
# !unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('python3 -m pip install visdom')
get_ipython().system_raw('python3 -m visdom.server -port 6006 >> visdomlog.txt 2>&1 &')
get_ipython().system_raw('ssh -o ServerAliveInterval=60 -o StrictHostKeyChecking=no -R YOURDOMAINHERE:80:localhost:6006 serveo.net &')
print('Visdom view: {}'.format('https://YOURDOMAINHERE.serveo.net/'))

Visdom view: https://YOURDOMAINHERE.serveo.net/


### utils.py

In [0]:
from    matplotlib import pyplot as plt
import  torch
from    torch import nn

class Flatten(nn.Module):

    def __init__(self):
        super(Flatten, self).__init__()

    def forward(self, x):
        shape = torch.prod(torch.tensor(x.shape[1:])).item() # 取c h w
        return x.view(-1, shape) # 占了第一个维度


def plot_image(img, label, name):

    fig = plt.figure()
    for i in range(6):
        plt.subplot(2, 3, i + 1)
        plt.tight_layout()
        plt.imshow(img[i][0]*0.3081+0.1307, cmap='gray', interpolation='none')
        plt.title("{}: {}".format(name, label[i].item()))
        plt.xticks([])
        plt.yticks([])
    plt.show()

### main.py

In [14]:
import  torch
from    torch.utils.data import DataLoader
from    torchvision import datasets
from    torchvision import transforms
from    torch import nn, optim
import  visdom
import  torchvision

from    torchvision.models import resnet18 # 现有模型
# from    utils import Flatten


batchsz = 128
lr = 1e-3
epochs = 5

device = torch.device('cuda')
torch.manual_seed(1234) # 随机种子，来保证实验可以复现出来
viz = visdom.Visdom(port='6006')


# 加载数据集
cifar_train = datasets.CIFAR10('cifar', True, transform=transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], # 数据归一化，R G B上的标准差
                            std=[0.229, 0.224, 0.225]) # R G B上的方标准差
]), download=True)
cifar_train = DataLoader(cifar_train, batch_size=batchsz, shuffle=True)

cifar_test = datasets.CIFAR10('cifar', False, transform=transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                            std=[0.229, 0.224, 0.225])
]), download=True)
cifar_test = DataLoader(cifar_test, batch_size=batchsz, shuffle=True)

x, label = iter(cifar_train).next() # 一个batch
print('x:', x.shape, 'label:', label.shape)
# x:torch.Size([32,3,32,32]) label:torch.Size([32])


# 评估ACC函数，验证或测试一样
def evalute(model, loader):
    model.eval() # test也是
    
    correct = 0 # 预测正确的数量
    total = len(loader.dataset)

    for x,y in loader:
        x,y = x.to(device), y.to(device)
        with torch.no_grad(): # 不需要反向传播
            logits = model(x)
            pred = logits.argmax(dim=1)
        correct += torch.eq(pred, y).sum().float().item() # item转为numpy

    return correct / total


def main():

    # 构建自定义网络
    # model = Lenet5().to(device)
    # model = ResNet18().to(device)
    # print(model) # 打印结构（初始化类里的结构，forward里打印不出来）

    # 构建迁移学习模型
    trained_model = resnet18(pretrained=True) # 使用已有模型训练好的各层参数（最后一层除外）
    model = nn.Sequential(*list(trained_model.children())[:-1], # 迁移，[b, 512, 1, 1]，去掉了最后一层
                          Flatten(), # 拉平c h w，[b, 512, 1, 1] => [b, 512]
                          nn.Linear(512, 10) # 新要学习的全连接层，5分类
                          ).to(device)
    # x = torch.randn(2, 3, 224, 224)
    # print(model(x).shape) # 迁移后[2,10]
    print(model)


    optimizer = optim.Adam(model.parameters(), lr=lr)
    criteon = nn.CrossEntropyLoss().to(device)

    best_acc, best_epoch = 0, 0
    global_step = 0 # x坐标
    viz.line([0], [-1], win='loss', opts=dict(title='loss'))
    viz.line([0], [-1], win='val_acc', opts=dict(title='val_acc'))

    for epoch in range(epochs):
        # 训练
        for step, (x,y) in enumerate(cifar_train): 
            
            x, y = x.to(device), y.to(device) # x: [b, 3, 224, 224], y: [b]
            
            model.train()
            logits = model(x)
            loss = criteon(logits, y) # CrossEntropy内部做one hot，直接传入y

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            viz.line([loss.item()], [global_step], win='loss', update='append')
            global_step += 1

        # 这打印的是此epoch中最后一个batch的loss，loss数据仅供参考，主要看TEST ACC
        print('epoch:', epoch, ' loss:', loss.item()) # loss是tensor标量，要通过item转为numpy打印出来

        # 验证
        if epoch % 1 == 0: 

            val_acc = evalute(model, cifar_test) # 评估验证集的ACC
            if val_acc > best_acc:
                best_epoch = epoch
                best_acc = val_acc

                torch.save(model.state_dict(), 'best.mdl') # 保存模型状态checkpoint，文件后缀无所谓

                viz.line([val_acc], [global_step], win='val_acc', update='append')
            
            print('epoch:', epoch, ' val acc:', val_acc) 

    print('val best epoch:', best_epoch, ' val best acc:', best_acc)

    # 加载checkpoint
    model.load_state_dict(torch.load('best.mdl')) # 加载模型checkpoint
    print('loaded from ckpt!')
    # best_acc = checkpoint['acc']
    # best_epoch = checkpoint['epoch']
    # print('checkpoint best acc:', best_acc, 'checkpoint best epoch:', best_epoch)

    # 测试
    test_acc = evalute(model, cifar_test) # 评估测试集的ACC（cifar没有分验证集和测试集，这里就用一样的）
    print('test acc:', test_acc)


if __name__ == '__main__':
    main()

Setting up a new session...


Files already downloaded and verified
Files already downloaded and verified
x: torch.Size([128, 3, 32, 32]) label: torch.Size([128])
Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNor